In [45]:
# importar bibliotecas de ciencias de dados
import pandas as pd
import numpy as np
import math
import statistics
from scipy import stats


# ler o arquivo csv agpop
df = pd.read_csv('agpop.csv', sep=',', index_col='Unnamed: 0')
df.head()

,county,state,acres92,acres87,acres82,farms92,farms87,farms82,largef92,largef87,largef82,smallf92,smallf87,smallf82,region
1,ALEUTIAN ISLANDS AREA,AK,683533.0,726596.0,764514.0,26,27,28,14,16,20,6,4,1,W
2,ANCHORAGE AREA,AK,47146.0,59297.0,256709.0,217,245,223,9,10,11,41,52,38,W
3,FAIRBANKS AREA,AK,141338.0,154913.0,204568.0,168,175,170,25,28,21,12,18,25,W
4,JUNEAU AREA,AK,210.0,214.0,127.0,8,8,12,0,0,0,5,4,8,W
5,KENAI PENINSULA AREA,AK,50810.0,85712.0,98035.0,93,119,137,9,18,17,12,18,19,W


Informações dos anos de 1982, 1987 e 1992 sobre:
- acres
- número de fazendas
- fazendas pequenas
- fazendas grandes
- região

Separada por cidade e estado.

Format
This data frame contains the following columns:

county:
county name (character variable)

state:
state abbreviation (character variable)

acres92:
number of acres devoted to farms, 1992

acres87:
number of acres devoted to farms, 1987

acres82:
number of acres devoted to farms, 1982

farms92:
number of farms, 1992

farms87:
number of farms, 1987

farms82:
number of farms, 1982

largef92:
number of farms with 1,000 acres or more, 1992

largef87:
number of farms with 1,000 acres or more, 1987

largef82:
number of farms with 1,000 acres or more, 1982

smallf92:
number of farms with 9 acres or fewer, 1992

smallf87:
number of farms with 9 acres or fewer, 1987

smallf82:
number of farms with 9 acres or fewer, 1982

region:
S = south; W = west; NC = north central; NE = northeast

## Análise e Tratamento

In [27]:
def print_is_duplicated(df):
    """
    Função que verifica se os dados estão duplicados. Se sim,
    mostra a quantidade de dados duplicados.
    """
    confirmation = df.duplicated().values.any()
    print(f"Existem valores duplicados? {confirmation}")
    if confirmation == True:
        print(f"Existem no total {df.duplicated().sum()} linhas duplicadas.")
    else:
        print('Não existem linhas duplicadas.')


def print_is_na(df):
    """
    Função que verifica se os dados estão duplicados. Se sim,
    retorna um dataframe com a quantidade de dados duplicados
    em valor absoluto e a porcentagem relativa.
    """
    confirmation = df.isnull().values.any()
    print(f"Existem valores nulos? {confirmation}\n")
    if confirmation == True:
        na = pd.concat([df.isna().sum(), round((df.isna().sum()/df.shape[0])*100, 2)], axis = 1).reset_index()
        na.columns = ["coluna", "absoluto", "porcentagem"]
#         print(na )
        return na
    else:
        print('Não existem valores ausentes.')

In [28]:
# ver se existem valores duplicados
print_is_duplicated(df)

Existem valores duplicados? False
Não existem linhas duplicadas.


In [29]:
# ver se existem valores nulos
df_na = print_is_na(df)
df_na

Existem valores nulos? True



,coluna,absoluto,porcentagem
0,county,0,0.00
1,state,0,0.00
2,acres92,19,0.62
3,acres87,23,0.75
4,acres82,17,0.55
5,farms92,0,0.00
6,farms87,0,0.00
7,farms82,0,0.00
8,largef92,0,0.00
9,largef87,0,0.00


In [30]:
# pegar df_na onde porcentagem é maior que 0
df_na = df_na[df_na['porcentagem'] > 0]

In [31]:
from scipy.stats import shapiro

df_normal = pd.DataFrame({
    'columns': df_na['coluna']
})

normal_list = []
for coluna in df_na['coluna']:
    try:
        _, p_value = shapiro(df[coluna])
        normal_list.append('Normal' if p_value > 0.05 else 'Não normal')
    except:
        normal_list.append('erro')
        
df_normal['normalidade'] = normal_list
df_normal

,columns,normalidade
2,acres92,Normal
3,acres87,Normal
4,acres82,Normal


In [32]:
# nome das colunas que passaram no teste da normalidade
list_col = df_normal.query("normalidade == 'Normal'")['columns'].unique().tolist()

# Imputação com a mediana para variáveis com alta proporção de dados ausentes
for col in list_col:
    df[col].fillna(df[col].median(), inplace=True)

# Amostragem

## Amostragem Aleatória Simples sem reposição

Estimar: <br>
(1) A  ́area total dedicada a fazendas em 1992,<br>
(2) A  ́area m ́edia dedicada a fazendas em 1992,<br>
(3) A propor ̧c ̃ao de condados com menos de 250.000 acres em fazendas em 1992,<br>
(4) O total de condados com menos de 250.000 acres em fazendas em 1992.<br>

Para cada tipo de estimação, obter:<br>
(1) Obtenha as estimativas, erro padr ̃ao e intervalo da confian ̧ca para cada um dos planos amostrais;<br>
(2) Fa ̧ca uma an ́alise comparativa dos resultados no item anterior, em termos do vi ́es, erro quadr ́atico
m ́edio e coeficiente de varia ̧c ̃ao<br>
(3) Considere uma estimativa da raz ̃ao e de regress ̃ao da  ́area total (media) dedicada a fazendas em
1992, considerando como vari ́avel auxiliar a  ́area total dedicada a fazendas em 1982 (acres82) e A
 ́area total dedicada a fazendas em 1987 (acres87).<br>
(4) Compare os resultados do item anterior em termos de precis ̃ao e variabilidade relativa.

In [33]:
def calcular_erro_padrao(amostra):
    # Tamanho da amostra (n)
    n = len(amostra)

    # Variância amostral
    variancia_amostral = statistics.variance(amostra)

    # Desvio padrão da amostra (s)
    desvio_padrao_amostral = math.sqrt(variancia_amostral)

    # Erro padrão (SE)
    erro_padrao = desvio_padrao_amostral / math.sqrt(n)

    return erro_padrao

In [43]:
def calcular_intervalo_confianca(amostra, nivel_confianca=0.95):
    # Remover valores ausentes (NaN) da amostra
    amostra = [x for x in amostra if not math.isnan(x)]

    # Tamanho da amostra (n)
    n = len(amostra)

    # Média amostral (estimativa da área média das fazendas)
    media_amostral = sum(amostra) / n

    # Desvio padrão da amostra
    desvio_padrao_amostral = statistics.stdev(amostra)

    # Graus de liberdade
    graus_liberdade = n - 1

    # Valor crítico t para o nível de confiança desejado
    valor_critico_t = stats.t.ppf((1 + nivel_confianca) / 2, graus_liberdade)

    # Erro padrão (SE)
    erro_padrao = desvio_padrao_amostral / math.sqrt(n)

    # Intervalo de confiança
    limite_inferior = media_amostral - valor_critico_t * erro_padrao
    limite_superior = media_amostral + valor_critico_t * erro_padrao

    return limite_inferior, limite_superior

In [34]:
# amostrar 500 linhas aleatórias do dataframe
df_aas = df.sample(n=500, random_state=42)

In [46]:
# estimar área total dedicada a fazendas em 1992
area_total = df_aas['acres92'].sum()
print('Estimação da área total em 1992: {:.2f} acres'.format(area_total))

# estimar erro padrão da área total
erro_padrao = calcular_erro_padrao(df_aas['acres92'])
print('Erro padrão da estimativa: {:.2f} acres'.format(erro_padrao))

# estimar intervalo de confiança da área total
limite_inferior, limite_superior = calcular_intervalo_confianca(df_aas['acres92'])
print('Intervalo de confiança: {:.2f} a {:.2f} acres'.format(limite_inferior, limite_superior))

Estimação da área total em 1992: 166807163.00 acres
Erro padrão da estimativa: 21534.51 acres
Intervalo de confiança: 291304.85 a 375923.81 acres


In [47]:
# estimar área média dedicada a fazendas em 1992
n=500 # amostra

n_farms = df_aas['farms92'].sum()

# divide a area_total pelo n_farms
area_media = area_total / n_farms
print('Estimação da área média em 1992: {:.2f} acres'.format(area_media))

# estimar erro padrão da área média
erro_padrao = calcular_erro_padrao(df_aas['acres92'] / df_aas['farms92'])
print('Erro padrão da estimativa: {:.2f} acres'.format(erro_padrao))

# estimar intervalo de confiança da área média
limite_inferior, limite_superior = calcular_intervalo_confianca(df_aas['acres92'] / df_aas['farms92'])
print('Intervalo de confiança: {:.2f} a {:.2f} acres'.format(limite_inferior, limite_superior))

Estimação da área média em 1992: 503.04 acres
Erro padrão da estimativa: 405.28 acres
Intervalo de confiança: 543.28 a 2135.80 acres


## Amostragem aleatória Estratificado por região (region) com alocação proporcional

## Amostragem Aleatória por conglomerados usando o estado (state) como a variável de cluster,